In [3]:
import pandas as pd
import numpy as np
import json

In [350]:
# Getting list of 100 industries from 10 sectors. 
import requests
import pandas as pd
import numpy as np
import sys
import json
df = pd.DataFrame()

def get_avg_sector_rates():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.groupby('SEC_CONV').head(100)
    df = df.iloc[:, [8,3]]
    df['MRKCAP'] = df['MRKCAP'].str.replace('$', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace(',', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace('M', ' M')
    df['MRKCAP'] = df['MRKCAP'].str.replace('B', ' B')
    df['SEC_CONV'] = df['SEC_CONV'].fillna(value='No Industry Identified')
    df['MRKCAP'] = df['MRKCAP'].fillna(value='0 D')
    df.dropna(inplace = True)
    df[['MRKCAP_AMT','MRKCAP_TYPE']] = df.MRKCAP.str.split(" ",expand=True,)
    millions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 100, fill_value = 5)
    billions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 1000000, fill_value = 5)
    df.loc[df['MRKCAP_TYPE'] == 'M', 'Conversion'] = millions
    df.loc[df['MRKCAP_TYPE'] == 'B', 'Conversion'] = billions
    df.loc[df['MRKCAP_TYPE'] == 'D', 'Conversion'] = 0
    df.loc[df['MRKCAP_TYPE'] == 'None', 'Conversion'] = df['MRKCAP_AMT'].fillna(value='0')
    df['Conversion'] = df['Conversion'].fillna(value='0')
    df['Conversion'] = df['Conversion'].astype(int)
    print(df.dtypes)
    print(sys.maxsize)
    group_sectors = df.groupby(['SEC_CONV','Conversion'])
    res = df.groupby(['SEC_CONV'])['Conversion'].mean().reset_index()
    print(res)
    res = res.to_json(orient='columns')
    res = json.dumps(res)
    res = res.replace('\\','')
    res = res.replace('"{','{')
    res = res.replace('}"','}')
#   res.to_csv(r'c:/development/python\mrkcap.csv')
    return res
get_avg_sector_rates()

SEC_CONV       object
MRKCAP         object
MRKCAP_AMT     object
MRKCAP_TYPE    object
Conversion      int32
dtype: object
9223372036854775807
                 SEC_CONV    Conversion
0        Basic Industries  7.511351e+07
1           Capital Goods  9.196153e+07
2       Consumer Industry  8.041427e+07
3                 Finance  5.635905e+07
4             Health Care  6.102823e+07
5           Miscellaneous  6.134793e+07
6  No Industry Identified  3.162555e+07
7        Public Utilities  2.797412e+07
8       Technology/Energy  7.244389e+07
9          Transportation  2.580533e+08


'{"SEC_CONV":{"0":"Basic Industries","1":"Capital Goods","2":"Consumer Industry","3":"Finance","4":"Health Care","5":"Miscellaneous","6":"No Industry Identified","7":"Public Utilities","8":"Technology/Energy","9":"Transportation"},"Conversion":{"0":75113514.4736842066,"1":91961525.0400000066,"2":80414268.9599999934,"3":56359050.0,"4":61028234.1599999964,"5":61347932.3298969045,"6":31625552.0,"7":27974124.8615384623,"8":72443885.2399999946,"9":258053276.8571428657}}'

In [144]:
#API Data Pull
import pandas as pd
import requests
# symbol = "YI"
def get_stock_data_from_alphavantage_by_symbol(symbol):
    Alpha_Advantage_API = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=" + symbol + "&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        data = data['bestMatches']
        df2 = pd.DataFrame.from_dict(data, orient='columns')
        df2.columns = ['SYM', 'CO.', 'Type', 'Country', 'LP-Time', 'LP-End-Time', 'UTC-Type', 'Dollar-Type', 'Last-List-Price']
        df2['SYM'] = df2['SYM'].str.strip()
        df2.loc[df2['SYM'] == str(symbol)]
        return df2
    else:
        print("API did not pull any data from alphavantage")
# Executes the function
get_stock_data_from_alphavantage_by_symbol(symbol)


NameError: name 'symbol' is not defined

In [116]:
#Merging data that is found within the Second Dataframe from the API
def merge_company_data_for_single_api(sym1, sym2):
#     sym1 = get_stock_data_from_alphavantage_by_symbol()
#     sym2 = get_company_data_from_csv()
    return sym1.merge(sym2)

200


,SYM,CO.,Type,Country,LP-Time,LP-End-Time,UTC-Type,Dollar-Type,Last-List-Price,LP,MRKCAP,YR,SEC,IND,SUMQTE
0,YIN,Yintech Investment Holdings Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.4000,4.95,$353.66M,2016,Finance,Investment Bankers/Brokers/Service,https://www.nasdaq.com/symbol/yin


In [178]:
#Info from API
import pandas as pd
import requests
def get_sector_stats():
    Alpha_Advantage_API_Sector = "https://www.alphavantage.co/query?function=SECTOR&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API_Sector)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        df3 = pd.DataFrame.from_dict(data, orient='columns')
        df3.columns = ['SEC', 'Real_Time', '1_Day_Performance', '5_Day_Performance', '1_Month_Performance', '3_Month_Performance', 'YTD_Performance', '1_Year_Performance', '3_Year_Performance', '5_Year_Performance', '10_Year_Performance']
#         df3['SEC'] = df3['SEC'].str.strip()
#         df2.loc[df2['SYM'] == str(symbol)]
        df3 = df3['SEC']
        df3 = df3.replace('NaN', '')
        return df3
    else:
        print("API did not pull any data from alphavantage")
get_sector_stats()

200


Communication Services                                              NaN
Consumer Discretionary                                              NaN
Consumer Staples                                                    NaN
Energy                                                              NaN
Financials                                                          NaN
Health Care                                                         NaN
Industrials                                                         NaN
Information               US Sector Performance (realtime & historical)
Information Technology                                              NaN
Last Refreshed                                   04:20 PM ET 05/23/2019
Materials                                                           NaN
Real Estate                                                         NaN
Utilities                                                           NaN
Name: SEC, dtype: object

In [60]:
#Articles via API
import pandas as pd
import requests
def get_stock_news():
    stock_news_api = "https://stocknewsapi.com/api/v1?tickers=YI&items=50&type=article&token=qqygybi5zike7wfmu21lawdnfndqhhi53ndszk83"
    resp = requests.get(stock_news_api)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        data = data['data']
        df4 = pd.DataFrame.from_dict(data, orient='columns')
        df4.columns = ['DATE', 'IMG_URL', 'NEWS_URL', 'Sentiment', 'Source_Name', 'Tags', 'Text', 'SYM', 'Title', 'Type']
        return df4
    else:
        print("API did not pull any data from alphavantage")
get_stock_news()

200


,DATE,IMG_URL,NEWS_URL,Sentiment,Source_Name,Tags,Text,SYM,Title,Type
0,"Thu, 16 May 2019 19:11:29 -0400",https://stocknewsapi.com/images/v1/g/4/transcr...,https://seekingalpha.com/article/4264778-111-i...,Neutral,Seeking Alpha,[CallScript],"111, Inc's (YI) CEO Junling Liu on Q1 2019 Res...",[YI],"111, Inc's (YI) CEO Junling Liu on Q1 2019 Res...",Article


In [72]:
#Web Scrap
import pandas as pd
import requests
from bs4 import BeautifulSoup
symbols = ['yi','bhf']
companyimages = []
nasdaq_website = "https://www.nasdaq.com/symbol/bhf"
page = requests.get(nasdaq_website)
soup = BeautifulSoup(page.text, 'html.parser')
review_text_elem = soup.find_all(class_='quote-symbol-icon')

for item in review_text_elem:
    companyimages.append(item)

companyimages
# review_text_elem



[<img alt="BHF" class="quote-symbol-icon" height="40" src="https://www.nasdaq.com/logos/BHF.gif" width="100"/>]